# LOAD u data

In [1]:
import pandas as pd
import socket, struct, time
import numpy as np
import matplotlib.pyplot as plt
import threading
from IPython.display import clear_output
import plotly.graph_objects as go
import numpy as np
import catboost as cb
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

mass = []
df = pd.DataFrame(columns=['column1', 'column2', 'column3', 'column4', 'column5', 'column6', 'column7', 'column8', 'column9', 'column10', 'column11', 'column12', 'column13', 'column14', 'column15', 'column16', 'column17', 'column18', 'column19', 'column20', 'column21', 'column22', 'column23', 'column24', 'column25'])

from catboost import CatBoostClassifier
model = CatBoostClassifier()  
#gbm_plus = model.load_model('positive_30.01.2022.cbm')
#gbm_negative = model.load_model('negative_29.01.2022.cbm')
#gbm_negative = gbm
#gbm_plus = gbm1

#gbm_combine = model.load_model('main.cbm')
gbm_combine = model.load_model('main_model_2.13.02.2022.cbm')

def plus_def(gbm, df):
    ans = gbm.predict(df)
    if ans[0][0] == 2:
        return 0
    else:
        return 1
    
def negative_def(gbm, df):
    ans = gbm.predict(df)
    if ans[0][0] == 2:
        return 0
    else:
        return -1
    
    
def combine_def(gbm, df):
    ans = gbm.predict(df)
    if ans[0][0] == 2:
        return 0
    elif ans[0][0] == 1:
        return -1
    else:
        return 1

In [2]:
import plotly.graph_objects as go
import plotly.graph_objs as go

data = [go.Bar(
            x=['prediction'],
            y=[-1.5]
    )]
layout = {'yaxis': {'range': [-2, 2]}}
fig = go.FigureWidget(data, layout)
fig

FigureWidget({
    'data': [{'type': 'bar', 'uid': '30ad5762-0d11-4799-b63b-bd9fd85a015c', 'x': ['prediction']…

In [ ]:
class TCPParser: # The script contains one main class which handles DSI-Streamer data packet parsing.

	def __init__(self, host, port):
		self.host = host
		self.port = port
		self.done = False
		self.data_log = b''
		self.latest_packets = []
		self.latest_packet_headers = []
		self.latest_packet_data = np.zeros((1,1))
		self.signal_log = np.zeros((1,20))
		self.time_log = np.zeros((1,20))
		self.montage = []
		self.fsample = 0
		self.fmains = 0

		self.sock = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
		self.sock.connect((self.host,self.port))
		
	def parse_data(self):
		
		# parse_data() receives DSI-Streamer TCP/IP packets and updates the signal_log and time_log attributes
		# which capture EEG data and time data, respectively, from the last 100 EEG data packets (by default) into a numpy array.
		while not self.done:
			data = self.sock.recv(921600)
			self.data_log += data
			if self.data_log.find(b'@ABCD',0,len(self.data_log)) != -1:										# The script looks for the '@ABCD' header start sequence to find packets.
				for index,packet in enumerate(self.data_log.split(b'@ABCD')[1:]):							# The script then splits the inbound transmission by the header start sequence to collect the individual packets.
					self.latest_packets.append(b'@ABCD' + packet)
				for packet in self.latest_packets:
					self.latest_packet_headers.append(struct.unpack('>BHI',packet[5:12]))
				self.data_log = b''


				for index, packet_header in enumerate(self.latest_packet_headers):		
					# For each packet in the transmission, the script will append the signal data and timestamps to their respective logs.
					if packet_header[0] == 1:
						if np.shape(self.signal_log)[0] == 1:												# The signal_log must be initialized based on the headset and number of available channels.
							self.signal_log = np.zeros((int(len(self.latest_packets[index][23:])/4),20))
							self.time_log = np.zeros((1,20))
							self.latest_packet_data = np.zeros((int(len(self.latest_packets[index][23:])/4),1))

						self.latest_packet_data = np.reshape(struct.unpack('>%df'%(len(self.latest_packets[index][23:])/4),self.latest_packets[index][23:]),(len(self.latest_packet_data),1))
						self.latest_packet_data_timestamp = np.reshape(struct.unpack('>f',self.latest_packets[index][12:16]),(1,1))

						#print("Timestamps: " + str(self.latest_packet_data_timestamp))
						#print("Signal Data: " + str(self.latest_packet_data))
						global mass
						mass = self.latest_packet_data
						self.signal_log = np.append(self.signal_log,self.latest_packet_data,1)
						self.time_log = np.append(self.time_log,self.latest_packet_data_timestamp,1)
						self.signal_log = self.signal_log[:,-100:]
						self.time_log = self.time_log[:,-100:]
					## Non-data packet handling
					if packet_header[0] == 5:
						(event_code, event_node) = struct.unpack('>II',self.latest_packets[index][12:20])
						if len(self.latest_packets[index])>24:
							message_length = struct.unpack('>I',self.latest_packets[index][20:24])[0]
						print("Event code = " + str(event_code) + "  Node = " + str(event_node))
						if event_code == 9:
							montage = self.latest_packets[index][24:24+message_length].decode()
							montage = montage.strip()
							print("Montage = " + montage)
							self.montage = montage.split(',')
						if event_code == 10:
							frequencies = self.latest_packets[index][24:24+message_length].decode()
							print("Mains,Sample = "+ frequencies)
							mains,sample = frequencies.split(',')
							self.fsample = float(sample)
							self.fmains = float(mains)
			self.latest_packets = []
			self.latest_packet_headers = []

	def example_plot(self):

		# example_plot() uses the threading Python library and matplotlib to plot the EEG data in realtime. 
		# The plots are unlabeled but users can refer to the TCP/IP Socket Protocol Documentation to understand how to discern the different plots given their indices.
		# Ideally, each EEG plot should have its own subplot but for demonstrative purposes, they are all plotted on the same figure.
		data_thread = threading.Thread(target=self.parse_data)
		data_thread.start()

		refresh_rate = 0.03
		duration = 60	# The default plot duration is 60 seconds.
		runtime = 0

		#fig = plt.figure()

		while True: # runtime < duration/refresh_rate:
			self.signal_log = self.signal_log[:,-1000:]
			self.time_log = self.time_log[:,-1000:]
			plt.clf()
			try:
				#print('mass')
				#print(mass.tolist())
				df = pd.DataFrame(columns=['column1', 'column2', 'column3', 'column4', 'column5', 'column6', 'column7', 'column8', 'column9', 'column10', 'column11', 'column12', 'column13', 'column14', 'column15', 'column16', 'column17', 'column18', 'column19', 'column20', 'column21', 'column22', 'column23', 'column24', 'column25'])
				to_append = [i[0] for i in mass.tolist()]
				a_series = pd.Series(to_append, index = df.columns)
				df = df.append(a_series, ignore_index=True)
				df.drop(['column25'], axis=1)
				#plus = plus_def(gbm_plus, df)
				#minus = negative_def(gbm_negative, df)
				#minus = negative_def(gbm_negative, df)
				#fig = go.Figure([go.Bar(x=['Negative impact','Positive impact',], y=[minus,plus])])
				#fig.update_layout(yaxis_range=[-2,2])
				#f2 = go.FigureWidget(fig)
				#clear_output(wait=False)
				#f2.show()
				base = combine_def(gbm_combine, df)
				bar = fig.data[0]
				bar.y = [base]
				#bar.y = [minus, plus]
				#print([minus, plus])
				#minus = model2.predict(df)
				#print(plus, minus)
				#print(plus[0][0], minus[0][0])
				#colors = ['blue','green']
				#fig = go.Figure(data=[go.Bar(x=['Negative', 'Positive'],y=[minus[0][0], plus[0][0]],marker_color=colors)])
				#fig.update_yaxes(range = [-4,4])
				#fig.update_layout(
				#	title="Negative vs Positive",
				#	font_family="Courier New",
				#	title_font_family="Times New Roman",
				#	font=dict(
				#		family="Courier New, monospace",
				#		size=24,
				#		color="black"))
				#clear_output(wait=True)
				#fig.show()
			except Exception as e:
				print(e)
			runtime += 1
		plt.show()
		self.done = True
		data_thread.join()

		
if __name__ == "__main__":

	# The script will automatically run the example_plot() method if not called from another script.
	tcp = TCPParser('localhost',8844)
	tcp.example_plot()


'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'


'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'


'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'


'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'


'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'


...

'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'


'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'


'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'


'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'
'list' object has no attribute 'tolist'


Event code = 3  Node = 1
Event code = 1  Node = 0
Event code = 9  Node = 1
Montage = P3,C3,F3,Fz,F4,C4,P4,Cz,CM,A1,Fp1,Fp2,T3,T5,O1,O2,X3,X2,F7,F8,X1,A2,T6,T4,TRG
Event code = 12  Node = 1
Event code = 10  Node = 1
Mains,Sample = 60,300
Event code = 2  Node = 1
Event code = 3  Node = 1
Event code = 1  Node = 0
Event code = 9  Node = 1
Montage = P3,C3,F3,Fz,F4,C4,P4,Cz,CM,A1,Fp1,Fp2,T3,T5,O1,O2,X3,X2,F7,F8,X1,A2,T6,T4,TRG
Event code = 12  Node = 1
Event code = 10  Node = 1
Mains,Sample = 60,300
Event code = 2  Node = 1
Event code = 3  Node = 1
Event code = 1  Node = 0
Event code = 9  Node = 1
Montage = P3,C3,F3,Fz,F4,C4,P4,Cz,CM,A1,Fp1,Fp2,T3,T5,O1,O2,X3,X2,F7,F8,X1,A2,T6,T4,TRG
Event code = 12  Node = 1
Event code = 10  Node = 1
Mains,Sample = 60,300
Event code = 2  Node = 1
Event code = 3  Node = 1
Event code = 1  Node = 0
Event code = 9  Node = 1
Montage = P3,C3,F3,Fz,F4,C4,P4,Cz,CM,A1,Fp1,Fp2,T3,T5,O1,O2,X3,X2,F7,F8,X1,A2,T6,T4,TRG
Event code = 12  Node = 1
Event code = 10  Node = 1

In [ ]:
for kk in range(0,len(mass)):
    to_append = [i[0] for i in mass[kk].tolist()]
    a_series = pd.Series(to_append, index = df.columns)
    df = df.append(a_series, ignore_index=True)
    
df['y+'] = 2
df['y-'] = -1

df.to_excel("part2/output12.xlsx")

# load data

In [7]:
%%time
import pandas as pd

df1 = pd.read_excel('part2/output1.xlsx', index_col=0) #1
df2 = pd.read_excel('part2/output2.xlsx', index_col=0) #1
df3 = pd.read_excel('part2/output3.xlsx', index_col=0) #1
df4 = pd.read_excel('part2/output4.xlsx', index_col=0) #1
df5 = pd.read_excel('part2/output5.xlsx', index_col=0) 
df6 = pd.read_excel('part2/output6.xlsx', index_col=0) 
df7 = pd.read_excel('part2/output7.xlsx', index_col=0) 
df8 = pd.read_excel('part2/output8.xlsx', index_col=0) 
df9 = pd.read_excel('part2/output9.xlsx', index_col=0) 
#df10 = pd.read_excel('part2/output10.xlsx', index_col=0) #3
#df11 = pd.read_excel('part2/output11.xlsx', index_col=0) #3
df12 = pd.read_excel('part2/output12.xlsx', index_col=0)
df13 = pd.read_excel('part2/output13.xlsx', index_col=0) 
df14 = pd.read_excel('part2/output14.xlsx', index_col=0) 
df15 = pd.read_excel('part2/output15.xlsx', index_col=0) 
df16 = pd.read_excel('part2/output16.xlsx', index_col=0)
df17 = pd.read_excel('part2/output17.xlsx', index_col=0)
df_out1 = pd.read_excel('part2/output_1+3-1.xlsx', index_col=0) #3
df_out2 = pd.read_excel('part2/output_2+3-1.xlsx', index_col=0) #3

Wall time: 4min 40s


In [8]:
df1_ = pd.read_excel('part2/check1.xlsx', index_col=0) 
df2_ = pd.read_excel('part2/check2.xlsx', index_col=0) 
df3_ = pd.read_excel('part2/check3.xlsx', index_col=0) 

In [42]:
frames = [df1,df4,df5,df6,df7,df8,df9,df12,df13,df14,df15,df16,df17,df_out1,df2,df3,df_out2,df1_,df2_,df3_]
result = pd.concat(frames)

res = [df2,df3,df_out2,df1_,df2_,df3_]
ress = pd.concat(res)
#ress.to_excel("part2/main_base_fool_validation.xlsx")

In [ ]:
new_y = result['y+']
new_X = result.drop(['y+', 'y-','column25'], axis=1)

In [ ]:
new_y_validation = ress['y+']
new_X_validation = ress.drop(['y+', 'y-','column25'], axis=1)

In [ ]:
params ={'loss_function': 'MultiClass', 'eval_metric': 'TotalF1', 'learning_rate': 0.04122092866861257, 'l2_leaf_reg': 0.3006158716462908, 'colsample_bylevel': 0.09856162056948979, 'depth': 8, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'min_data_in_leaf': 3, 'one_hot_max_size': 8}

In [ ]:
gbm = cb.CatBoostClassifier(**params)
gbm.fit(new_X, new_y, eval_set=[(new_X_validation, new_y_validation)], verbose=1)

In [ ]:
import pandas as pd
import numpy as np

df_describe = pd.DataFrame(new_y)
df_describe.describe()
df_describe.hist()

In [32]:
r1 = pd.read_excel('part2/check1.xlsx', index_col=0) 
y = r1['y+']
X = r1.drop(['y+', 'y-','column25'], axis=1)

f1_score(y.tolist(), gbm.predict(pd.DataFrame(X)).tolist(), average='weighted')

1.0

In [33]:
r2 = pd.read_excel('part2/check2.xlsx', index_col=0) 
y = r2['y+']
X = r2.drop(['y+', 'y-','column25'], axis=1)

f1_score(y.tolist(), gbm.predict(pd.DataFrame(X)).tolist(), average='weighted')

1.0

In [34]:
r3 = pd.read_excel('part2/check3.xlsx', index_col=0) 
y = r3['y+']
X = r3.drop(['y+', 'y-','column25'], axis=1)

f1_score(y.tolist(), gbm.predict(pd.DataFrame(X)).tolist(), average='weighted')

1.0

In [35]:
res5 = [r1,r2,r3]
ress5 = pd.concat(res5)

y = ress5['y+']
X = ress5.drop(['y+', 'y-','column25'], axis=1)

f1_score(y.tolist(), gbm.predict(X).tolist(), average='weighted')

1.0

In [ ]:
gbm.save_model('main_model.04.02.2022.cbm')

In [ ]:
#065 087 081

In [37]:
import os
import pandas as pd

dirname = 'C:/Users/lysyi/Desktop/oscillofraph/part2/global_check'
files = os.listdir(dirname)
print(files)

['out_123_21.xlsx', 'out_124_31.xlsx', 'out_142_13.xlsx', 'out_145_21.xlsx', 'out_153_31.xlsx', 'out_161_31.xlsx', 'out_165_31.xlsx', 'out_233_31.xlsx', 'out_241_13.xlsx', 'out_242_31.xlsx', 'out_251_13.xlsx', 'out_262_31.xlsx', 'out_264_13.xlsx', 'out_341_31.xlsx', 'out_346_31.xlsx', 'out_353_21.xlsx', 'out_411_13.xlsx', 'out_433_13.xlsx', 'out_442_31.xlsx', 'out_464_31.xlsx', 'out_466_13.xlsx', 'out_512_21.xlsx', 'out_513_13.xlsx', 'out_525_31.xlsx', 'out_543_21.xlsx', 'out_551_21.xlsx', 'out_554_31.xlsx', 'out_612_21.xlsx', 'out_621_13.xlsx', 'out_626_31.xlsx', 'out_635_31.xlsx', 'out_645_31.xlsx', 'out_653_21.xlsx', 'out_655_21.xlsx', 'out_662_31.xlsx', 'out_664_31.xlsx', 'out_666_31.xlsx']


In [38]:
r = [pd.read_excel(dirname+'/'+i, index_col=0) for i in files]

In [39]:
frames = [df1,df4,df5,df6,df7,df8,df9,df12,df13,df14,df15,df16,df17,df_out1,df2,df3,df_out2,df1_,df2_,df3_]

for k in r:
    frames.append(k)
result = pd.concat(frames)

In [41]:
#gbm = model.load_model('main.cbm')

y = result['y+'].astype('int')
X = result.drop(['y+', 'y-','column25'], axis=1)

#y_validation = ress['y+']
#X_validation = ress.drop(['y+', 'y-','column25'], axis=1)

In [42]:
import catboost as cb

gbm = model.load_model('main.cbm')

#params ={'loss_function': 'MultiClass', 'eval_metric': 'TotalF1', 'learning_rate': 0.04122092866861257, 'l2_leaf_reg': 0.3006158716462908, 'colsample_bylevel': 0.09856162056948979, 'depth': 8, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'min_data_in_leaf': 3, 'one_hot_max_size': 8}
#gbm = cb.CatBoostClassifier(**params)
gbm.fit(X, y, eval_set=[(X, y)], verbose=1)

0:	learn: 0.8402926	test: 0.8402926	best: 0.8402926 (0)	total: 429ms	remaining: 7m 8s
1:	learn: 0.9110871	test: 0.9110871	best: 0.9110871 (1)	total: 884ms	remaining: 7m 21s
2:	learn: 0.8964656	test: 0.8964656	best: 0.9110871 (1)	total: 1.25s	remaining: 6m 53s
3:	learn: 0.9024375	test: 0.9024375	best: 0.9110871 (1)	total: 1.65s	remaining: 6m 49s
4:	learn: 0.8999779	test: 0.8999779	best: 0.9110871 (1)	total: 2.04s	remaining: 6m 45s
5:	learn: 0.9054404	test: 0.9054404	best: 0.9110871 (1)	total: 2.47s	remaining: 6m 49s
6:	learn: 0.9101625	test: 0.9101625	best: 0.9110871 (1)	total: 2.86s	remaining: 6m 45s
7:	learn: 0.9037001	test: 0.9037001	best: 0.9110871 (1)	total: 3.23s	remaining: 6m 40s
8:	learn: 0.9103844	test: 0.9103844	best: 0.9110871 (1)	total: 3.66s	remaining: 6m 43s
9:	learn: 0.9077151	test: 0.9077151	best: 0.9110871 (1)	total: 3.89s	remaining: 6m 25s
10:	learn: 0.9022350	test: 0.9022350	best: 0.9110871 (1)	total: 4.18s	remaining: 6m 15s
11:	learn: 0.9096580	test: 0.9096580	best: 

93:	learn: 0.9489030	test: 0.9489030	best: 0.9489030 (93)	total: 37.1s	remaining: 5m 57s
94:	learn: 0.9490163	test: 0.9490163	best: 0.9490163 (94)	total: 37.6s	remaining: 5m 58s
95:	learn: 0.9491727	test: 0.9491727	best: 0.9491727 (95)	total: 38.2s	remaining: 5m 59s
96:	learn: 0.9490884	test: 0.9490884	best: 0.9491727 (95)	total: 38.5s	remaining: 5m 58s
97:	learn: 0.9489780	test: 0.9489780	best: 0.9491727 (95)	total: 38.8s	remaining: 5m 56s
98:	learn: 0.9491496	test: 0.9491496	best: 0.9491727 (95)	total: 39.2s	remaining: 5m 56s
99:	learn: 0.9492182	test: 0.9492182	best: 0.9492182 (99)	total: 39.5s	remaining: 5m 55s
100:	learn: 0.9494863	test: 0.9494863	best: 0.9494863 (100)	total: 40s	remaining: 5m 55s
101:	learn: 0.9497749	test: 0.9497749	best: 0.9497749 (101)	total: 40.4s	remaining: 5m 55s
102:	learn: 0.9499357	test: 0.9499357	best: 0.9499357 (102)	total: 40.8s	remaining: 5m 55s
103:	learn: 0.9498926	test: 0.9498926	best: 0.9499357 (102)	total: 41s	remaining: 5m 53s
104:	learn: 0.950

184:	learn: 0.9526586	test: 0.9526586	best: 0.9526713 (182)	total: 1m 13s	remaining: 5m 22s
185:	learn: 0.9526474	test: 0.9526474	best: 0.9526713 (182)	total: 1m 13s	remaining: 5m 22s
186:	learn: 0.9526457	test: 0.9526457	best: 0.9526713 (182)	total: 1m 13s	remaining: 5m 21s
187:	learn: 0.9526662	test: 0.9526662	best: 0.9526713 (182)	total: 1m 14s	remaining: 5m 21s
188:	learn: 0.9526945	test: 0.9526945	best: 0.9526945 (188)	total: 1m 14s	remaining: 5m 21s
189:	learn: 0.9526966	test: 0.9526966	best: 0.9526966 (189)	total: 1m 15s	remaining: 5m 21s
190:	learn: 0.9527173	test: 0.9527173	best: 0.9527173 (190)	total: 1m 15s	remaining: 5m 21s
191:	learn: 0.9527237	test: 0.9527237	best: 0.9527237 (191)	total: 1m 16s	remaining: 5m 21s
192:	learn: 0.9527108	test: 0.9527108	best: 0.9527237 (191)	total: 1m 16s	remaining: 5m 21s
193:	learn: 0.9527147	test: 0.9527147	best: 0.9527237 (191)	total: 1m 17s	remaining: 5m 20s
194:	learn: 0.9527208	test: 0.9527208	best: 0.9527237 (191)	total: 1m 17s	remain

274:	learn: 0.9530060	test: 0.9530060	best: 0.9530072 (273)	total: 1m 47s	remaining: 4m 43s
275:	learn: 0.9529962	test: 0.9529962	best: 0.9530072 (273)	total: 1m 47s	remaining: 4m 43s
276:	learn: 0.9529860	test: 0.9529860	best: 0.9530072 (273)	total: 1m 48s	remaining: 4m 43s
277:	learn: 0.9529855	test: 0.9529855	best: 0.9530072 (273)	total: 1m 48s	remaining: 4m 42s
278:	learn: 0.9529845	test: 0.9529845	best: 0.9530072 (273)	total: 1m 49s	remaining: 4m 42s
279:	learn: 0.9529883	test: 0.9529883	best: 0.9530072 (273)	total: 1m 49s	remaining: 4m 42s
280:	learn: 0.9529883	test: 0.9529883	best: 0.9530072 (273)	total: 1m 50s	remaining: 4m 42s
281:	learn: 0.9529956	test: 0.9529956	best: 0.9530072 (273)	total: 1m 50s	remaining: 4m 41s
282:	learn: 0.9530010	test: 0.9530010	best: 0.9530072 (273)	total: 1m 51s	remaining: 4m 41s
283:	learn: 0.9530009	test: 0.9530009	best: 0.9530072 (273)	total: 1m 51s	remaining: 4m 40s
284:	learn: 0.9530073	test: 0.9530073	best: 0.9530073 (284)	total: 1m 51s	remain

364:	learn: 0.9531068	test: 0.9531068	best: 0.9531106 (362)	total: 2m 23s	remaining: 4m 9s
365:	learn: 0.9531004	test: 0.9531004	best: 0.9531106 (362)	total: 2m 23s	remaining: 4m 8s
366:	learn: 0.9531002	test: 0.9531002	best: 0.9531106 (362)	total: 2m 23s	remaining: 4m 8s
367:	learn: 0.9531023	test: 0.9531023	best: 0.9531106 (362)	total: 2m 24s	remaining: 4m 7s
368:	learn: 0.9531231	test: 0.9531231	best: 0.9531231 (368)	total: 2m 24s	remaining: 4m 7s
369:	learn: 0.9531191	test: 0.9531191	best: 0.9531231 (368)	total: 2m 25s	remaining: 4m 7s
370:	learn: 0.9531232	test: 0.9531232	best: 0.9531232 (370)	total: 2m 25s	remaining: 4m 6s
371:	learn: 0.9531211	test: 0.9531211	best: 0.9531232 (370)	total: 2m 25s	remaining: 4m 6s
372:	learn: 0.9531213	test: 0.9531213	best: 0.9531232 (370)	total: 2m 26s	remaining: 4m 6s
373:	learn: 0.9531196	test: 0.9531196	best: 0.9531232 (370)	total: 2m 26s	remaining: 4m 5s
374:	learn: 0.9531295	test: 0.9531295	best: 0.9531295 (374)	total: 2m 27s	remaining: 4m 5s

...

634:	learn: 0.9531842	test: 0.9531842	best: 0.9531842 (634)	total: 4m 14s	remaining: 2m 26s
635:	learn: 0.9531822	test: 0.9531822	best: 0.9531842 (634)	total: 4m 15s	remaining: 2m 26s
636:	learn: 0.9531822	test: 0.9531822	best: 0.9531842 (634)	total: 4m 15s	remaining: 2m 25s
637:	learn: 0.9531798	test: 0.9531798	best: 0.9531842 (634)	total: 4m 15s	remaining: 2m 25s
638:	learn: 0.9531806	test: 0.9531806	best: 0.9531842 (634)	total: 4m 16s	remaining: 2m 24s
639:	learn: 0.9531812	test: 0.9531812	best: 0.9531842 (634)	total: 4m 16s	remaining: 2m 24s
640:	learn: 0.9531789	test: 0.9531789	best: 0.9531842 (634)	total: 4m 17s	remaining: 2m 23s
641:	learn: 0.9531772	test: 0.9531772	best: 0.9531842 (634)	total: 4m 17s	remaining: 2m 23s
642:	learn: 0.9531772	test: 0.9531772	best: 0.9531842 (634)	total: 4m 18s	remaining: 2m 23s
643:	learn: 0.9531764	test: 0.9531764	best: 0.9531842 (634)	total: 4m 18s	remaining: 2m 22s
644:	learn: 0.9531764	test: 0.9531764	best: 0.9531842 (634)	total: 4m 19s	remain

724:	learn: 0.9531715	test: 0.9531715	best: 0.9531842 (634)	total: 4m 59s	remaining: 1m 53s
725:	learn: 0.9531703	test: 0.9531703	best: 0.9531842 (634)	total: 4m 59s	remaining: 1m 53s
726:	learn: 0.9531663	test: 0.9531663	best: 0.9531842 (634)	total: 5m	remaining: 1m 52s
727:	learn: 0.9531663	test: 0.9531663	best: 0.9531842 (634)	total: 5m 1s	remaining: 1m 52s
728:	learn: 0.9531657	test: 0.9531657	best: 0.9531842 (634)	total: 5m 1s	remaining: 1m 52s
729:	learn: 0.9531657	test: 0.9531657	best: 0.9531842 (634)	total: 5m 1s	remaining: 1m 51s
730:	learn: 0.9531659	test: 0.9531659	best: 0.9531842 (634)	total: 5m 2s	remaining: 1m 51s
731:	learn: 0.9531663	test: 0.9531663	best: 0.9531842 (634)	total: 5m 3s	remaining: 1m 50s
732:	learn: 0.9531663	test: 0.9531663	best: 0.9531842 (634)	total: 5m 3s	remaining: 1m 50s
733:	learn: 0.9531663	test: 0.9531663	best: 0.9531842 (634)	total: 5m 3s	remaining: 1m 50s
734:	learn: 0.9531669	test: 0.9531669	best: 0.9531842 (634)	total: 5m 4s	remaining: 1m 49s


814:	learn: 0.9531717	test: 0.9531717	best: 0.9531842 (634)	total: 5m 41s	remaining: 1m 17s
815:	learn: 0.9531717	test: 0.9531717	best: 0.9531842 (634)	total: 5m 42s	remaining: 1m 17s
816:	learn: 0.9531715	test: 0.9531715	best: 0.9531842 (634)	total: 5m 42s	remaining: 1m 16s
817:	learn: 0.9531700	test: 0.9531700	best: 0.9531842 (634)	total: 5m 43s	remaining: 1m 16s
818:	learn: 0.9531698	test: 0.9531698	best: 0.9531842 (634)	total: 5m 43s	remaining: 1m 15s
819:	learn: 0.9531721	test: 0.9531721	best: 0.9531842 (634)	total: 5m 44s	remaining: 1m 15s
820:	learn: 0.9531730	test: 0.9531730	best: 0.9531842 (634)	total: 5m 44s	remaining: 1m 15s
821:	learn: 0.9531726	test: 0.9531726	best: 0.9531842 (634)	total: 5m 45s	remaining: 1m 14s
822:	learn: 0.9531730	test: 0.9531730	best: 0.9531842 (634)	total: 5m 46s	remaining: 1m 14s
823:	learn: 0.9531715	test: 0.9531715	best: 0.9531842 (634)	total: 5m 47s	remaining: 1m 14s
824:	learn: 0.9531726	test: 0.9531726	best: 0.9531842 (634)	total: 5m 47s	remain

905:	learn: 0.9531646	test: 0.9531646	best: 0.9531842 (634)	total: 6m 26s	remaining: 40.1s
906:	learn: 0.9531634	test: 0.9531634	best: 0.9531842 (634)	total: 6m 27s	remaining: 39.7s
907:	learn: 0.9531634	test: 0.9531634	best: 0.9531842 (634)	total: 6m 27s	remaining: 39.3s
908:	learn: 0.9531636	test: 0.9531636	best: 0.9531842 (634)	total: 6m 28s	remaining: 38.8s
909:	learn: 0.9531636	test: 0.9531636	best: 0.9531842 (634)	total: 6m 28s	remaining: 38.4s
910:	learn: 0.9531634	test: 0.9531634	best: 0.9531842 (634)	total: 6m 28s	remaining: 38s
911:	learn: 0.9531627	test: 0.9531627	best: 0.9531842 (634)	total: 6m 29s	remaining: 37.6s
912:	learn: 0.9531635	test: 0.9531635	best: 0.9531842 (634)	total: 6m 30s	remaining: 37.2s
913:	learn: 0.9531630	test: 0.9531630	best: 0.9531842 (634)	total: 6m 30s	remaining: 36.8s
914:	learn: 0.9531633	test: 0.9531633	best: 0.9531842 (634)	total: 6m 31s	remaining: 36.4s
915:	learn: 0.9531626	test: 0.9531626	best: 0.9531842 (634)	total: 6m 32s	remaining: 36s
916

996:	learn: 0.9531627	test: 0.9531627	best: 0.9531842 (634)	total: 7m 6s	remaining: 1.28s
997:	learn: 0.9531634	test: 0.9531634	best: 0.9531842 (634)	total: 7m 7s	remaining: 856ms
998:	learn: 0.9531644	test: 0.9531644	best: 0.9531842 (634)	total: 7m 7s	remaining: 428ms
999:	learn: 0.9531639	test: 0.9531639	best: 0.9531842 (634)	total: 7m 8s	remaining: 0us

bestTest = 0.9531842399
bestIteration = 634

Shrink model to first 635 iterations.


In [43]:
gbm.save_model('main_model_3.13.02.2022.cbm')

# dd

# MODEL LEARNING

In [ ]:
%%time

import pandas as pd
new_df = pd.read_excel('part2/main_base4_wo2.xlsx', index_col=0)  

In [ ]:
new_df = result

new_y_plus = new_df['y+']
new_X = new_df.drop(['y+', 'y-','column25'], axis=1)
new_X

In [ ]:
import numpy as np
import catboost as cb
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import optuna

def objective(trial):
    train_x, train_y =new_X,new_y_plus

    param = {
        "loss_function": trial.suggest_categorical("loss_function", ["MultiClass"]),
        "eval_metric": trial.suggest_categorical("eval_metric", ["Accuracy"]),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e0),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-2, 1e0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 8, 16),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 2, 20),
        "one_hot_max_size": trial.suggest_int("one_hot_max_size", 2, 20),  
    }

    gbm = cb.CatBoostClassifier(**param)

    gbm.fit(train_x, train_y, eval_set=[(X, y_check)], verbose=1, early_stopping_rounds=100)

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)
    pred_labels = [int(i) for i in pred_labels]
    f1 = f1_score(valid_y, pred_labels,average='weighted')

    return f1

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective)

In [ ]:
params ={'loss_function': 'MultiClass', 'eval_metric': 'TotalF1', 'learning_rate': 0.004122092866861257, 'l2_leaf_reg': 0.3006158716462908, 'colsample_bylevel': 0.09856162056948979, 'depth': 8, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'min_data_in_leaf': 3, 'one_hot_max_size': 8}

In [ ]:
import numpy as np
import catboost as cb
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
#import optuna

gbm1 = cb.CatBoostClassifier(**params,iterations=300)
 
new_y_plus = result['y+']
new_X = result.drop(['y+', 'y-','column25'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(new_X.to_numpy(), list(new_y_plus), random_state=42, shuffle = True)

#e = check_df['y+']
#new_y_negative = new_df['y-']
#ee = check_df.drop(['y+', 'y-','column25'], axis=1)


gbm1.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=2, early_stopping_rounds=50, use_best_model=True)

In [ ]:
gbm.save_model('positive_02.02.2022.cbm')

In [ ]:
gbm_plus = gbm

In [ ]:
from sklearn.metrics import f1_score

base = pd.read_excel('part2/output6.xlsx', index_col=0) 
e = base['y+']
ee = base.drop(['y+', 'y-','column25'], axis=1)

In [ ]:
f1_score(gbm_negative.predict(ee), e.tolist(), average='weighted')

In [ ]:
import pandas as pd
import numpy as np

df_describe = pd.DataFrame(new_y)
df_describe.describe()
df_describe.hist()

In [ ]:
gbm.save_model('positive_model5.cbm')

# Model for positive impact

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_X.to_numpy(), list(new_y_plus), random_state=42, shuffle = True)

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=10, depth=9, learning_rate=0.03) # классификатор
model.fit(X_train, y_train) # обучение классификатора
#model.score(X_test, y_test)

In [ ]:
ress = model2.predict(df)
ress2 = [i[0] for i in ress.tolist()]

### Качество на тесте

In [ ]:
model.score(X_test, y_test)

### Качество на трейне

In [ ]:
model.score(X_train, y_train)

### Predict

# Model for negative impact

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_X.to_numpy(), list(new_y_negative), random_state=42, shuffle = True)

In [ ]:
from catboost import CatBoostClassifier

model2 = CatBoostClassifier(iterations=70, depth=9, learning_rate=0.03) # классификатор
model2.fit(X_train, y_train) # обучение классификатора
model2.score(X_test, y_test)

### Качество на тесте

In [ ]:
model2.score(X_test, y_test)

### Качество на трейне

In [ ]:
model2.score(X_train, y_train)

### Predict

In [ ]:
ress = model2.predict(df)
ress2 = [i[0] for i in ress.tolist()]